In [1]:
# Either you can store the  OpenAI key in the “OPENAI_API_KEY” environment variable.
# or pass it here as below from a config.ini
import configparser
workingFolder=r'C:\Users\jfrancis\OneDrive - GalaxE. Solutions, Inc\GalaxE D Drive\AI Journey\Gen AI'
# Read the configuration file
config = configparser.ConfigParser()
config.read(workingFolder+'\\config.ini')
OPENAI_API_KEY=config.get('General','OPENAI_API_KEY')
ACTIVELOOP_TOKEN=config.get('General','ACTIVELOOP_TOKEN')
ACTIVELOOP_ORG_ID=config.get('General','ACTIVELOOP_ORG_ID')
HUGGINGFACEHUB_API_TOKEN=config.get('General','HUGGINGFACEHUB_API_TOKEN')
GOOGLE_API_KEY=config.get('General','GOOGLE_API_KEY')
GOOGLE_CSE_ID=config.get('General','GOOGLE_CSE_ID')

<img src="https://images.spr.so/cdn-cgi/imagedelivery/j42No7y-dcokJuNgXeA0ig/e2ea6e36-749a-4e11-a123-1482dc4ad4e2/Screenshot_2023-06-09_at_14.00.40/w=1920,quality=80" />

We set up the environment and retrieved the news article.

    Install required libraries: The first step is to ensure that the necessary libraries, namely requests, newspaper3k, and LangChain, are installed.
    Scrape articles: We will use the requests library to scrape the content of the target news articles from their respective URLs.
    Extract titles and text: The newspaper library will be used to parse the scraped HTML, extracting the titles and text of the articles.
    Preprocess the text: The extracted texts need to be cleaned and preprocessed to make them suitable for input to LLM.

The rest will explore new possibilities to enhance the application’s performance further.

    Use Few-Shot Learning Technique: We use the few-shot learning technique in this step. This template will provide a few examples of the language model to guide it in generating the summaries in the desired format - a bulleted list.
    Generate summaries: With the modified prompt, we utilize the model to generate concise summaries of the extracted articles' text in the desired format.
    Use the Output Parsers: We employ the Output Parsers to interpret the output from the language model, ensuring it aligns with our desired structure and format.
    Output the results: Finally, we present the bulleted summaries along with the original titles, enabling users to quickly grasp the main points of each article in a structured manner.

In [3]:
import requests
from newspaper import Article

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.82 Safari/537.36'
}

article_url = "https://www.artificialintelligence-news.com/2022/01/25/meta-claims-new-ai-supercomputer-will-set-records/"

session = requests.Session()

try:
    response = session.get(article_url, headers=headers, timeout=10)
  
    if response.status_code == 200:
        article = Article(article_url)
        article.download()
        article.parse()
      
        print(f"Title: {article.title}")
        print(f"Text: {article.text}")
    else:
        print(f"Failed to fetch article at {article_url}")
except Exception as e:
    print(f"Error occurred while fetching article at {article_url}: {e}")

Title: Meta claims its new AI supercomputer will set records
Text: Ryan is a senior editor at TechForge Media with over a decade of experience covering the latest technology and interviewing leading industry figures. He can often be sighted at tech conferences with a strong coffee in one hand and a laptop in the other. If it's geeky, he’s probably into it. Find him on Twitter (@Gadget_Ry) or Mastodon (@gadgetry@techhub.social)

Meta (formerly Facebook) has unveiled an AI supercomputer that it claims will be the world’s fastest.

The supercomputer is called the AI Research SuperCluster (RSC) and is yet to be fully complete. However, Meta’s researchers have already begun using it for training large natural language processing (NLP) and computer vision models.

RSC is set to be fully built in mid-2022. Meta says that it will be the fastest in the world once complete and the aim is for it to be capable of training models with trillions of parameters.

“We hope RSC will help us build entire

In [4]:
#Few Shot Prompting
from langchain.schema import (
    HumanMessage
)

# we get the article data from the scraping part
article_title = article.title
article_text = article.text

# prepare template for prompt
template = """
As an advanced AI, you've been tasked to summarize online articles into bulleted points. Here are a few examples of how you've done this in the past:

Example 1:
Original Article: 'The Effects of Climate Change
Summary:
- Climate change is causing a rise in global temperatures.
- This leads to melting ice caps and rising sea levels.
- Resulting in more frequent and severe weather conditions.

Example 2:
Original Article: 'The Evolution of Artificial Intelligence
Summary:
- Artificial Intelligence (AI) has developed significantly over the past decade.
- AI is now used in multiple fields such as healthcare, finance, and transportation.
- The future of AI is promising but requires careful regulation.

Now, here's the article you need to summarize:

==================
Title: {article_title}

{article_text}
==================

Please provide a summarized version of the article in a bulleted list format.
"""

# Format the Prompt
prompt = template.format(article_title=article.title, article_text=article.text)

messages = [HumanMessage(content=prompt)]

Here we have a few important components:

Article data: The title and text of the article are obtained, which will be used as inputs to the model.

Template preparation: A template is prepared for the prompt. This template includes a few-shot learning style, where the model is provided with examples of how it has previously converted articles into a bulleted list format. The template also includes placeholders for the actual article title and text that will be summarized.  Then, the placeholders in the template ({article_title} and {article_text}) are replaced with the actual title and text of the article using the .format() method. 

The next step is to use ChatOpenAI class to load the GPT-3.5-turbo model for generating the summary. Then, the formatted prompt is passed to the language model as the input/prompt. The ChatOpenAI class's chat instance takes a HumanMessage list as an input argument.

In [7]:
from langchain.chat_models import ChatOpenAI
import os
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY

# load the model
chat = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.0)

# generate summary
summary = chat(messages)
print(summary.content)

- Meta (formerly Facebook) has unveiled an AI supercomputer called the AI Research SuperCluster (RSC).
- The RSC is expected to be the world's fastest supercomputer once complete.
- It will be capable of training models with trillions of parameters.
- Meta aims to use the RSC to build new AI systems for real-time voice translations and applications in the metaverse.
- The RSC is estimated to be 20x faster than Meta's current clusters and 9x faster at running the NVIDIA Collective Communication Library (NCCL).
- It will also be 3x faster at training large-scale natural language processing (NLP) workflows.
- Meta designed the RSC with security and privacy controls to use real-world examples from its production systems.
- This will allow Meta to advance research for tasks such as identifying harmful content on its platforms.
- The RSC is expected to significantly improve training times for models with tens of billions of parameters.


In [8]:
# Output Parsers
from langchain.output_parsers import PydanticOutputParser
from pydantic import validator
from pydantic import BaseModel, Field
from typing import List


# create output parser class
class ArticleSummary(BaseModel):
    title: str = Field(description="Title of the article")
    summary: List[str] = Field(description="Bulleted list summary of the article")

    # validating whether the generated summary has at least three lines
    @validator('summary', allow_reuse=True)
    def has_three_or_more_lines(cls, list_of_lines):
        if len(list_of_lines) < 3:
            raise ValueError("Generated summary has less than three bullet points!")
        return list_of_lines

# set up output parser
parser = PydanticOutputParser(pydantic_object=ArticleSummary)

The next step involves creating a template for the input prompt that instructs the language model to summarize the news article into bullet points. This template is used to instantiate a PromptTemplate object, which is responsible for correctly formatting the prompts that are sent to the language model. The PromptTemplate uses our custom parser to format the prompt sent to the language model using the .get_format_instructions() method, which will include additional instructions on how the output should be shaped.

In [9]:
from langchain.prompts import PromptTemplate


# create prompt template
# notice that we are specifying the "partial_variables" parameter
template = """
You are a very good assistant that summarizes online articles.

Here's the article you want to summarize.

==================
Title: {article_title}

{article_text}
==================

{format_instructions}
"""

prompt = PromptTemplate(
    template=template,
    input_variables=["article_title", "article_text"],
    partial_variables={"format_instructions": parser.get_format_instructions()}
)

# Format the prompt using the article title and text obtained from scraping
formatted_prompt = prompt.format_prompt(article_title=article_title, article_text=article_text)

Lastly, the GPT-3 model with the temperature set to 0.0  is initialized, which means the output will be deterministic, favoring the most likely outcome over randomness/creativity. The parser object then converts the string output from the model to a defined schema using the .parse() method.

In [10]:
from langchain.llms import OpenAI

# instantiate model class
model = OpenAI(model_name="text-davinci-003", temperature=0.0)

# Use the model to generate a summary
output = model(formatted_prompt.to_string())

# Parse the output into the Pydantic model
parsed_output = parser.parse(output)
print(parsed_output)

title='Meta claims its new AI supercomputer will set records' summary=['Meta (formerly Facebook) has unveiled an AI supercomputer called the AI Research SuperCluster (RSC) that it claims will be the world’s fastest.', 'RSC is set to be fully built in mid-2022 and is expected to be capable of training models with trillions of parameters.', 'Meta expects RSC will be 20x faster than its current V100-based clusters and is estimated to be 9x faster at running the NVIDIA Collective Communication Library (NCCL) and 3x faster at training large-scale NLP workflows.', 'RSC was designed with security and privacy controls in mind to allow Meta to use real-world examples from its production systems in production training.', 'Meta says that its previous AI research infrastructure only leveraged open source and other publicly-available datasets.']


The Pydantic output parser is a powerful method for molding and structuring the output from language models. It uses the Pydantic library, known for its data validation capabilities, to define and enforce data schemas for the model's output.

This is a recap of what we did:

    We defined a Pydantic data structure named ArticleSummary. This model serves as a blueprint for the desired structure of the generated article summary. It comprises fields for the title and the summary, which is expected to be a list of strings representing bullet points. Importantly, we incorporate a validator within this model to ensure the summary comprises at least three points, thereby maintaining a certain level of detail in the summarization.
    We then instantiate a parser object using our ArticleSummary class. This parser plays a crucial role in ensuring the output generated by the language model aligns with the defined structures of our custom schema.
    To direct the language model's output, we create the prompt template. The template instructs the model to act as an assistant that summarizes online articles by incorporating the parser object.
    So, output parsers enable us to specify the desired format of the model's output, making extracting meaningful information from the model's responses easier.